In [1]:
import random
import time
import datetime

import tensorflow as tf
import numpy as np
import scipy.misc
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data
slim = tf.contrib.slim

In [2]:
mnist = input_data.read_data_sets('MNIST_data')
x_size, y_size = 28, 28
n_classes = 10
default_collection = 'nodes'

def timestamp():
    d = datetime.datetime.now()
    return d.strftime("%Y/%m/%d/%X")

timestamp()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


'2016/12/10/21:27:49'

In [3]:
def random_batch_iterator(x, y, *, batch_size):
    n = x.shape[0]
    assert n == y.shape[0]
    
    while True:
        index = np.random.randint(n, size=batch_size)
        x_batch, y_batch = x[index], y[index]
        yield x_batch.copy(), y_batch.copy()
        
def batch_iterator(x, y, batch_size):
    n = x.shape[0]
    assert n == y.shape[0]
    
    for i in range(0, n, batch_size):
        x_batch, y_batch = x[i:i+batch_size], y[i:i+batch_size]        
        yield x_batch.copy(), y_batch.copy()

In [4]:
def build_cnn(inputs, *, is_training, n_conv, conv_base, conv_mul,
              conv_size, pool_size, collection=default_collection):
    l = inputs
    for i in range(n_conv):
        n_filters = conv_base * conv_mul ** i
        l = slim.conv2d(l, n_filters, [conv_size, conv_size],
                        scope='Conv{}'.format(i+1), outputs_collections=collection)
        l = slim.max_pool2d(l, [pool_size, pool_size], scope='MaxPool{}'.format(i+1),
                            outputs_collections=collection)
    l = slim.flatten(l)
    
    l = slim.dropout(l, 0.5, scope='Dropout', is_training=is_training,
                     outputs_collections=collection)
    l = slim.fully_connected(l, 10, activation_fn=None, scope='Output',
                             outputs_collections=collection)
    return l

def build_loss(logits, y_true):
    logloss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y_true))
    return logloss

In [5]:
batch_size = 512

n_conv = 2
conv_base = 32
conv_mul = 2
conv_size = 5
pool_size = 2

def build_model(graph, build_cnn):
    with graph.as_default():#, graph.device('/cpu:0'):
        with tf.variable_scope('model') as vs:
            is_training = tf.placeholder(tf.bool)
            x_ph = tf.placeholder(tf.float32, shape=[batch_size, x_size * y_size])
            x_image = tf.reshape(x_ph, [-1, x_size, y_size, 1])
            y_ph = tf.placeholder(tf.int64, shape=[batch_size])

            logits = build_cnn(x_image, is_training=is_training, n_conv=n_conv,
                               conv_base=conv_base, conv_mul=conv_mul,
                               conv_size=conv_size, pool_size=pool_size)

            prediction = tf.nn.softmax(logits)

            loss = build_loss(logits, y_ph)

            optimizer = tf.train.AdamOptimizer().minimize(loss)

            correct_prediction = tf.equal(tf.argmax(prediction, 1), y_ph)
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        # Code to use of tensorboard
        with tf.name_scope('summaries'):
            tf.scalar_summary('log_loss', loss)
            tf.scalar_summary('acc', accuracy)
            merged_summary = tf.merge_all_summaries()
            
    return {
        'is_training': is_training,
        'x_ph': x_ph,
        'y_ph': y_ph,
        'prediction': prediction,
        'loss': loss,
        'optimizer': optimizer,
        'accuracy': accuracy,
        'merged_summary': merged_summary
    }

In [6]:
n_epochs = 1000

def train_net(layers, n_epochs):
    l = layers
    train_iterator = random_batch_iterator(mnist.train.images, mnist.train.labels, batch_size=batch_size)
    val_iterator = random_batch_iterator(mnist.validation.images, mnist.validation.labels, batch_size=batch_size)
    test_iterator = batch_iterator(mnist.test.images, mnist.test.labels, batch_size=batch_size)
    
    best_acc = 0.0
    path = '/tmp/tf/' + timestamp()
    with tf.Session(graph=graph) as session:
        #tf.train.export_meta_graph('metagraph')    
        train_writer = tf.train.SummaryWriter(
            path+'/train', session.graph)
        val_writer = tf.train.SummaryWriter(
            path+'/val', session.graph)
        saver = tf.train.Saver()
        tf.global_variables_initializer().run()

        train_operations = [l['optimizer'], l['merged_summary'], l['accuracy'], l['loss']]
        val_operations = [l['merged_summary'], l['accuracy'], l['loss']]
        for epoch in range(n_epochs):
            x_batch, y_batch = next(train_iterator)
            
            feed_dict = {l['x_ph']: x_batch, l['y_ph']: y_batch, l['is_training']: True}
            _, summary, acc, _ = session.run(train_operations, feed_dict)
            train_writer.add_summary(summary, epoch)

            x_batch, y_batch = next(val_iterator)
            feed_dict = {l['x_ph']: x_batch, l['y_ph']: y_batch, l['is_training']: False}
            summary, acc, _ = session.run(val_operations, feed_dict)
            val_writer.add_summary(summary, epoch)

            if acc > best_acc:
                best_acc = acc
                path = saver.save(session, 'model.ckpt')
                
        test_acc = 0
        n = 0
        for x_batch, y_batch in test_iterator:
            if len(x_batch) != batch_size:
                break
            feed_dict = {l['x_ph']: x_batch, l['y_ph']: y_batch, l['is_training']: False}
            test_acc += l['accuracy'].eval(feed_dict=feed_dict)
            n += 1
        test_acc = test_acc / n
    return test_acc

In [7]:
graph = tf.Graph()
layers = build_model(graph, build_cnn)

start = time.time()
acc = train_net(layers=layers, n_epochs=n_epochs)
end = time.time()

print('Acc: {}\ntime: {}'.format(acc, end - start))

Acc: 0.990953947368421
time: 49.76647353172302


In [8]:
def build_cnn_method2(inputs, *, is_training, n_conv, conv_base, conv_mul,
              conv_size, pool_size, collection=default_collection):
    l = inputs
    for i in range(n_conv):
        n_filters = conv_base * conv_mul ** i
        l = slim.conv2d(l, n_filters, [conv_size, 1],
                        scope='Conv{}_a'.format(i+1), outputs_collections=collection)
        l = slim.conv2d(l, n_filters, [1, conv_size],
                        scope='Conv{}_b'.format(i+1), outputs_collections=collection)
        
        l = slim.max_pool2d(l, [pool_size, pool_size], scope='MaxPool{}'.format(i+1),
                            outputs_collections=collection)
    l = slim.flatten(l)
    
    l = slim.dropout(l, 0.5, scope='Dropout', is_training=is_training,
                     outputs_collections=collection)
    l = slim.fully_connected(l, 10, activation_fn=None, scope='Output',
                             outputs_collections=collection)
    return l

In [9]:
graph = tf.Graph()
layers = build_model(graph, build_cnn_method2)

start = time.time()
acc = train_net(layers=layers, n_epochs=n_epochs)
end = time.time()

print('Acc: {}\ntime: {}'.format(acc, end - start))

Acc: 0.990234375
time: 65.5129086971283


In [10]:
def build_cnn_method1(inputs, *, is_training, n_conv, conv_base, conv_mul,
              conv_size, pool_size, collection=default_collection):
    l = inputs
    for i in range(n_conv):
        n_filters = conv_base * conv_mul ** i
        l = slim.conv2d(l, n_filters * 2 // 3, [conv_size, conv_size],
                        scope='Conv{}_a'.format(i+1), outputs_collections=collection)
        l = slim.conv2d(l, n_filters, [1, 1],
                        scope='Conv{}_b'.format(i+1), outputs_collections=collection)
        
        l = slim.max_pool2d(l, [pool_size, pool_size], scope='MaxPool{}'.format(i+1),
                            outputs_collections=collection)
    l = slim.flatten(l)
    
    l = slim.dropout(l, 0.5, scope='Dropout', is_training=is_training,
                     outputs_collections=collection)
    l = slim.fully_connected(l, 10, activation_fn=None, scope='Output',
                             outputs_collections=collection)
    return l

In [11]:
graph = tf.Graph()
layers = build_model(graph, build_cnn_method1)

start = time.time()
acc = train_net(layers=layers, n_epochs=n_epochs)
end = time.time()

print('Acc: {}\ntime: {}'.format(acc, end - start))

Acc: 0.9904399671052632
time: 118.781986951828


In [17]:
def build_cnn_method1sc(inputs, *, is_training, n_conv, conv_base, conv_mul,
              conv_size, pool_size, collection=default_collection):
    l = inputs
    for i in range(n_conv):
        n_filters = conv_base * conv_mul ** i
        l = slim.conv2d(l, n_filters, [conv_size, 1],
                        scope='Conv{}_a'.format(i+1), outputs_collections=collection)
        print(l.get_shape())
        l = slim.separable_conv1d(l, n_filters, [1, conv_size],
                        scope='Conv{}_b'.format(i+1), outputs_collections=collection)
        print(l.get_shape())
        l = slim.max_pool2d(l, [pool_size, pool_size], scope='MaxPool{}'.format(i+1),
                            outputs_collections=collection)
    l = slim.flatten(l)
    
    l = slim.dropout(l, 0.5, scope='Dropout', is_training=is_training,
                     outputs_collections=collection)
    l = slim.fully_connected(l, 10, activation_fn=None, scope='Output',
                             outputs_collections=collection)
    return l

In [18]:
graph = tf.Graph()
layers = build_model(graph, build_cnn_method1sc)

start = time.time()
acc = train_net(layers=layers, n_epochs=n_epochs)
end = time.time()

print('Acc: {}\ntime: {}'.format(acc, end - start))

(512, 28, 28, 32)
(512, 28, 6, 32)
(512, 14, 3, 64)
(512, 14, 1, 64)


ValueError: Negative dimension size caused by subtracting 2 from 1 for 'model/MaxPool2/MaxPool' (op: 'MaxPool') with input shapes: [512,14,1,64].